# Neural Ordinary Differential Equations 神经常微分方程

[翻译自原文](https://nbviewer.org/github/urtrial/neural_ode/blob/master/Neural%20ODEs.ipynb)

**过程**很大策程度上可以用微分方程来描述:可以是物理系统的演化、病人的医疗条件、市场的基本属性等。这些数据在本质上是序列化并连续的，这意味着对它们的观测仅仅是某些不断变化的状态的具象化表示。

还有另一种类型的序列数据是离散的——例如NLP数据:它的状态是离散的，从一个符号到另一个符号，从一个单词到另一个单词。

今天，这两种类型数据的处理通常使用循环神经网络。然而，它们在本质上是不同的，似乎应该区别对待。

NIPS2018有一篇非常有趣的[论文](https://arxiv.org/abs/1806.07366)试图解决这个问题。作者提出了一个非常有前途的方法，他们称之为**神经常微分方程**。

在这里，我试图复制和总结原始论文的结果，使你更容易熟悉这个想法。我相信，在卷积和循环网络中，这种新的结构可能很快就会出现在很多数据科学家的工具箱中。

想象一个问题:有一个过程跟随一个未知的ODE，并随着时间有一些关于这个过程的(有噪声的)观测
$$ \frac{dz}{dt} = f(z(t), t) \tag{1} $$$$ \{(z_0, t_0),(z_1, t_1),...,(z_M, t_M)\} - \text{observations} $$
那是否有可能找到动力学函数（dynamics function）$f(z, t)$的近似$\widehat{f}(z, t, \theta)$呢？

`ps: 较少自由度时，对那些从微分方程出发的相关问题，当强调微分方程本身，并集中讨论方程解的性质，这时常用「Dynamics」。Dynamics研究的是系统状态、输入、和输出随时间变化的规律 ----知乎`

首先考虑一个稍微简单点的任务：只有两个观测量，分别在轨迹的开始和结束处，$(z_0,t_0),(z_1,t_1)$。系统从$
(z_0,t_0)$开始通过使用任意ODE初值solver的参数化动态函数变化$t_1-t_0$的时间，结束时到达了新的状态$(\hat{z_1},t_1)$，我们希望通过改变参数$\theta$来最小化$\hat{z}_1$和观测$z_1$的差。 

或者，更形式化地说，考虑优化以下损失函数$L(\hat{z_1})$：
$$
L(z(t_1)) = L \Big( \int_{t_0}^{t_1} f(z(t), t, \theta)dt \Big) = L \big( \text{ODESolve}(z(t_0), f, t_0, t_1, \theta) \big) \tag{2}
$$


<img src=https://raw.githubusercontent.com/ziran-dean/picbed/main/images202201062246991.png width=50% />

<p style="text-align: center"> 
    图1：梯度的连续反向传播需要及时的反向求解增广ODE。<br /> 
    箭头表示通过来自观测的梯度来调节反向传播的梯度。<br />
    图片来自论文原文
</p>

如果你不想深入研究数学，那么上面的图展示了发生的事情。黑色的轨迹表示在前向传播时求解ODE。红色的箭头在反向传播时求解伴随ODE。

为了优化$L$，我们需要计算关于参数$z(t_0), t_0, t_1, \theta$的梯度。为了做到这点，我们需要确定loss是如何依赖于每个时刻的状态$z(t)$。
$$
a(t) = -\frac{\partial L}{\partial z(t)} \tag{3}
$$
$a(t)$ 被称之为**“伴随”（adjoint）** ，他的动力学依据另一个ODE，它可以被认为是链式法则的瞬时模拟。

$$
\frac{d a(t)}{d t} = -a(t) \frac{\partial f(z(t), t, \theta)}{\partial z} \tag{4}
$$
实际上这个公式的推导可以在原论文的附录中找到。

这里所有的向量都被认为是行向量，而原文中行列向量都用到了。

然后就可以计算：
$$
\frac{\partial L}{\partial z(t_0)} = \int_{t_1}^{t_0} a(t) \frac{\partial f(z(t), t, \theta)}{\partial z} dt \tag{5}
$$